In [1]:
import pandas as pd
import json 
import os
from json.decoder import JSONDecodeError
import shutil
pd.options.display.max_rows = None
pd.options.display.max_columns = None
path = "."
dados=[]
for file in os.scandir(path):
    if file.stat().st_size > 1024 and file.path.endswith(".json") and file.is_file():
        with open(file) as f:
            try:
                data1= json.loads("[" + f.read().replace("}\n{", "},{") + "]")
                app = pd.json_normalize(data1[0]['data']['attributes'])
                last_stats = data1[0]['data']['attributes']['last_analysis_stats']
                d = {'sha256': app['sha256'].values[0], 'malicious': last_stats['malicious'],'undetected': last_stats['undetected'], 'harmless': last_stats['harmless'] ,'suspicious': last_stats['suspicious'] ,'failure': last_stats['failure'],'unsupported': last_stats['type-unsupported'],'timeout': last_stats['timeout'],'confirmed_timeout': last_stats['confirmed-timeout']}
                dados.append(d)
                f.close()
    
            except JSONDecodeError as e:
                shutil.move(file,"Erros")
            except TypeError as e:
                shutil.move(file,"Erros")
df_VT = pd.DataFrame(dados)
df_VT.to_csv('out.csv', index=False) 
#for k, v in data1[0]["data"]["attributes"]["last_analysis_results"].items():
    #print("{:<30} {:<30}".format(k, str(v["result"])))
df1 = pd.read_csv('out.csv', sep = ',')
df1['TOTAL'] = df1.loc[df1['malicious'] >=0,['malicious','undetected','harmless','suspicious','failure','unsupported','timeout']].sum(axis=1)
df1['TOTAL'].fillna(0, inplace=True)

#with open(file+".labeled", "w") as file:
#    file.write()

print("Arquivos JSONs processados: ",len(df1))
df1.head()

Processados 3


,sha256,malicious,undetected,harmless,suspicious,failure,unsupported,timeout,confirmed_timeout,TOTAL
0,028f26390a79c1722f56e730a4aab5019ffca4a39b5782...,0,65,0,0,0,11,0,0,76
1,25706aa52cd3fa51753ba2957cceda34d22aa084c00a14...,1,58,0,0,0,11,4,1,74
2,2570e36a4b90b60d2d35d367ec29f33e77b0f851d5b332...,0,63,0,0,1,12,0,0,76
